First, let's import the necessary libraries and load the dataset:

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 2s 0us/step


Let's go on to defining the starting point for our transfer learning framework. A VGG16 model that has already been trained on the ImageNet dataset will be used.

In [2]:
base_model = keras.applications.VGG16(
    weights="imagenet",  
    input_shape=(32, 32, 3),
    include_top=False,
)  
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


We can now add some additional layers on top of the base model, and train the entire model end-to-end:

In [ ]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history1 = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

for layer in base_model.layers[:15]:
    layer.trainable = True

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history2 = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1563/1563 [==============================] - 742s 474ms/step - loss: 1.4882 - accuracy: 0.4755 - val_loss: 1.2750 - val_accuracy: 0.5507
Epoch 2/10
 873/1563 [===============>..............] - ETA: 4:32 - loss: 1.3183 - accuracy: 0.5374